In [1]:
from distutils.command.clean import clean

import pandas as pd
pd.options.display.float_format = '{:.0f}'.format
from pandas.api.types import CategoricalDtype
import numpy as np
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
import json
import statistics
import re
from sklearn.impute import KNNImputer
from decimal import Decimal

# For sentiment analysis of text
import nltk
#nltk.download("all") # Only first time
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


## December 2023

In [2]:
pd.read_csv("data/2023dic/d_listings.csv")

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6623,https://www.airbnb.com/rooms/6623,20231212042056,2023-12-12,city scrape,Rental unit in Venice · ★4.94 · 2 bedrooms · 4...,NaN,Close by is the Frari Church (known as Tiziano...,https://a0.muscache.com/pictures/492258/834683...,15016,...,5,5,5,NaN,f,3,3,0,0,1
1,6624,https://www.airbnb.com/rooms/6624,20231212042056,2023-12-12,city scrape,Rental unit in Venice · ★4.98 · 2 bedrooms · 6...,NaN,We are in the middle of a residential area cal...,https://a0.muscache.com/pictures/65440024/a564...,15016,...,5,5,5,NaN,f,3,3,0,0,1
2,12074,https://www.airbnb.com/rooms/12074,20231212042056,2023-12-12,city scrape,Rental unit in Venice · ★4.94 · 2 bedrooms · 4...,NaN,"There are plenty of bars, restaurants and pizz...",https://a0.muscache.com/pictures/11646330/65e7...,15016,...,5,5,5,NaN,f,3,3,0,0,1
3,27116,https://www.airbnb.com/rooms/27116,20231212042056,2023-12-12,city scrape,Bed and breakfast in Venice · ★4.84 · 1 bedroo...,NaN,"The area is very beautiful and characteristic,...",https://a0.muscache.com/pictures/miso/Hosting-...,116144,...,5,5,5,NaN,t,3,1,2,0,3
4,44527,https://www.airbnb.com/rooms/44527,20231212042056,2023-12-12,city scrape,Rental unit in Venice · ★4.85 · 3 bedrooms · 3...,NaN,"Cannaregio is a well-connected, truly Venetian...",https://a0.muscache.com/pictures/d81fb136-a867...,120215,...,5,5,5,M0270422294,f,2,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7881,1043303544124336952,https://www.airbnb.com/rooms/1043303544124336952,20231212042056,2023-12-12,city scrape,Rental unit in Venice · ★New · 1 bedroom · 1 b...,NaN,"Sestiere Cannaregio, vicino alla Chiesa San L...",https://a0.muscache.com/pictures/miso/Hosting-...,10072655,...,NaN,NaN,NaN,NaN,t,14,14,0,0,NaN
7882,1043366341069554128,https://www.airbnb.com/rooms/1043366341069554128,20231212042056,2023-12-12,city scrape,Hotel in Venice · ★New · 2 bedrooms · 2 beds ·...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,499474233,...,NaN,NaN,NaN,NaN,t,2,0,2,0,NaN
7883,1043440099679319097,https://www.airbnb.com/rooms/1043440099679319097,20231212042056,2023-12-12,city scrape,Rental unit in Venice · ★New · 2 bedrooms · 2 ...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,10072655,...,NaN,NaN,NaN,NaN,t,14,14,0,0,NaN
7884,1043464242138231179,https://www.airbnb.com/rooms/1043464242138231179,20231212042056,2023-12-12,city scrape,Rental unit in Venice · ★New · 1 bedroom · 1 b...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,10072655,...,NaN,NaN,NaN,NaN,t,14,14,0,0,NaN


In [3]:
dic2023_reviews = pd.read_csv("data/2023dic/reviews.csv")
dic2023_calendar = pd.read_csv("data/2023dic/calendar.csv", dtype={"listing_id": str,
                                                   "date": str,
                                                   "available": str,
                                                   "price": str,
                                                   "adjusted_price": str,
                                                   "minimum_nights": str,
                                                   "maximum_nights": str})
dic2023_listings = pd.read_csv("data/2023dic/listings.csv")
dic2023_neighbourhoods = pd.read_csv("data/2023dic/neighbourhoods.csv")
# dic2023_geo_neighbourhoods  # GeoJson
dic2023_d_listings = pd.read_csv("data/2023dic/d_listings.csv")
dic2023_d_reviews = pd.read_csv("data/2023dic/d_reviews.csv")


## Listings

In [4]:
dic2023_d_listings = dic2023_d_listings[['id', 'name', 'neighborhood_overview', 'host_id', 'host_since', 'host_location', 'host_about',
                   'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
                   'host_listings_count', 'host_total_listings_count', 'host_verifications',
                   'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed',
                   'latitude', 'longitude', 'property_type',
                   'accommodates', 'bathrooms_text', 'beds', 'price',
                   'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 'availability_30',
                   'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 'first_review',
                   'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                   'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
                   'review_scores_value', 'calculated_host_listings_count',
                   'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
                   'calculated_host_listings_count_shared_rooms', 'reviews_per_month']]


In [5]:
dic2023_d_listings.neighborhood_overview = dic2023_d_listings.neighborhood_overview.fillna("")
dic2023_d_listings.host_about = dic2023_d_listings.host_about.fillna("")

We can consider the distance between the Host house and the actual property

In [6]:
#location_geo = {}
#for l in tqdm(dic2023_d_listings.host_location.unique().tolist()):
#    geolocator = Nominatim(user_agent="Host to listing distance")
#    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.1)
#    host_location = geolocator.geocode(l)
#    location_geo[l] = (host_location.latitude, host_location.longitude)
#    
#with open("data/2023dic/hosts_locations.json", 'w') as f:
#    json.dump(location_geo, f)
#

### Host and listing location

In [7]:
with open("data/2023dic/hosts_locations.json", 'r') as f:
    location_geo = json.load(f)

dic2023_d_listings["host_location"] = dic2023_d_listings["host_location"].apply(lambda x: location_geo.get(x))

def geodesic_distancer(row):
    try:
        coords_1 = (row['host_location'][0], row['host_location'][1])
        coords_2 = (row["latitude"], row["longitude"])
        return geodesic(coords_1, coords_2).km
    except:
        return None

dic2023_d_listings['host_to_listing_geodesic_km'] = dic2023_d_listings.apply(geodesic_distancer, axis=1)
dic2023_d_listings.host_to_listing_geodesic_km = dic2023_d_listings.host_to_listing_geodesic_km.fillna(statistics.mode(dic2023_d_listings["host_to_listing_geodesic_km"]))
dic2023_d_listings.drop("host_location", axis=1, inplace=True)

### Response time

In [8]:
dic2023_d_listings.host_response_time = dic2023_d_listings.host_response_time.fillna("MISSING")
categorial_response_time = CategoricalDtype(categories= ['MISSING',
                                                         'a few days or more',
                                                         'within a day',
                                                         'within a few hours',
                                                         'within an hour'],
                                            ordered=True)
dic2023_d_listings.host_response_time = dic2023_d_listings.host_response_time.astype(categorial_response_time)

### Response rate

In [9]:
dic2023_d_listings["host_response_rate"] = dic2023_d_listings["host_response_rate"].str.rstrip('%').astype(float)
dic2023_d_listings.host_response_rate = dic2023_d_listings.host_response_rate.fillna(statistics.mode(dic2023_d_listings["host_response_rate"]))


### Acceptance rate

In [10]:
dic2023_d_listings["host_acceptance_rate"] = dic2023_d_listings["host_acceptance_rate"].str.rstrip('%').astype(float)
dic2023_d_listings.host_acceptance_rate = dic2023_d_listings.host_acceptance_rate.fillna(statistics.mode(dic2023_d_listings["host_acceptance_rate"]))


### Is superhost

In [11]:
dic2023_d_listings.host_is_superhost = dic2023_d_listings.host_is_superhost.fillna(statistics.mode(dic2023_d_listings["host_is_superhost"]))
dic2023_d_listings.host_is_superhost = dic2023_d_listings.host_is_superhost.apply(lambda x: 1 if x=="t" else 0)


### Bathrooms and bathrooms_text

In [12]:
dic2023_d_listings.bathrooms_text = dic2023_d_listings.bathrooms_text.fillna(statistics.mode(dic2023_d_listings["bathrooms_text"]))

#### Create bathrooms feature

In [13]:
def extract_digits(text):
    if "half" in text.lower():
        return '0.5'
    digits = re.findall(r'\d+\.\d+|\d+', text)
    return ''.join(digits)

dic2023_d_listings["bathrooms"] = dic2023_d_listings["bathrooms_text"].apply(extract_digits)
dic2023_d_listings["bathrooms"] = dic2023_d_listings["bathrooms"].astype(float)

#### Edit bathrooms text_feature

In [14]:
def remove_digits(text):
    return re.sub(r'\d', '', text).strip()
dic2023_d_listings["bathrooms_text"] = dic2023_d_listings["bathrooms_text"].apply(remove_digits)

remap_baths = {
    'baths': 'single',
    'bath': 'single',
    'private bath': 'private',
    'shared bath': 'shared',
    'shared baths': 'shared',
    'Shared half-bath': 'shared',
    '. baths': 'single',
    '. shared baths': 'shared',
    'Half-bath': 'single',
    'Private half-bath': 'private'
}

dic2023_d_listings["bathrooms_text"] = dic2023_d_listings["bathrooms_text"].replace(remap_baths)


### Beds

In [15]:
dic2023_d_listings.beds = dic2023_d_listings.beds.fillna(statistics.mode(dic2023_d_listings["beds"]))

### Availability

In [16]:
dic2023_d_listings.has_availability = dic2023_d_listings.has_availability.fillna(statistics.mode(dic2023_d_listings["has_availability"]))

## Dealing with Types before NAs imputation

In [17]:
dic2023_d_listings["host_id"] = dic2023_d_listings["host_id"].astype(str)
dic2023_d_listings["id"] = dic2023_d_listings["id"].astype(str)
dic2023_d_listings["host_since"] = pd.to_datetime(dic2023_d_listings["host_since"])


In [18]:
dic2023_d_listings["email_verification"] = False
dic2023_d_listings["phone_verification"] = False
dic2023_d_listings["work_email_verification"] = False

def allocate_verifications_to_variables(row):
    if "email" in row["host_verifications"]:
        row["email_verification"] = True
    if "phone" in row["host_verifications"]:
        row["phone_verification"] = True
    if "work_email" in row["host_verifications"]:
        row["work_email_verification"] = True
    return row

dic2023_d_listings = dic2023_d_listings.apply(allocate_verifications_to_variables, axis=1)

In [19]:
dic2023_d_listings.host_has_profile_pic = dic2023_d_listings.host_has_profile_pic.apply(lambda x: 1 if x=="t" else 0)

In [20]:
dic2023_d_listings.host_identity_verified = dic2023_d_listings.host_identity_verified.apply(lambda x: 1 if x=="t" else 0)

In [21]:
new_neighbourhoods_levels = {'Cannaregio': 'Centro Storico',
                             'San Marco':'Centro Storico',
                             'Isola San Giorgio': 'Centro Storico',
                             'San Polo':'Centro Storico',
                             'Castello': 'Centro Storico',
                             "Sant'Elena": 'Centro Storico',
                             'Dorsoduro': 'Centro Storico',
                             'Sacca Fisola': 'Centro Storico',
                             'Giudecca': 'Centro Storico',
                             'Tronchetto': 'Centro Storico',
                             'Santa Croce': 'Centro Storico',
                             "Ca' Emiliani": 'Terraferma',
                             'Marghera Zona Industriale': 'Terraferma',
                             'Marghera Catene': 'Terraferma',
                             'Marghera': 'Terraferma',
                             "Ca' Sabbioni":'Terraferma',
                             'Giustizia': 'Terraferma',
                             'San Lorenzo XXV Aprile': 'Terraferma',
                             'Bissuola': 'Terraferma',
                             'Cipressina': 'Terraferma',
                             'Zona Commerciale via Torino': 'Terraferma',
                             'Carpenedo': 'Terraferma',
                             'Villabona': 'Terraferma',
                             'Santa Barbara': 'Terraferma',
                             'Altobello': 'Terraferma',
                             'Piave 1860': 'Terraferma',
                             'La Favorita': 'Terraferma',
                             'Villaggio Sartori': 'Terraferma',
                             'Villaggio San Marco': 'Terraferma',
                             'Gazzera': 'Terraferma',
                             'Asseggiano': 'Terraferma',
                             "Pra' Secco": 'Terraferma',
                             'Gatta - Bondu?': 'Terraferma',
                             'Quartiere Pertini': 'Terraferma',
                             'Campalto CEP': 'Terraferma',
                             'Mestre': 'Terraferma',
                             "Scaramuzza": "Terraferma",
                             'Alberoni': 'Isole',
                             'Malamocco': 'Isole',
                             'Lido': 'Isole',
                             "Sant'Erasmo": 'Isole',
                             'Burano': 'Isole',
                             'San Pietro in Volta': 'Isole',
                             'Mazzorbo': 'Isole',
                             'Pellestrina': 'Isole',
                             'Murano': 'Isole',
                             'Torcello': 'Isole',
                             'Favaro': 'Terraferma',
                             'Case Dosa': 'Terraferma',
                             'Marocco Terraglio': 'Terraferma',
                             'Campalto Gobbi': 'Terraferma',
                             'Malcontenta': 'Terraferma',
                             'Zelarino': 'Terraferma',
                             'Chirignago': 'Terraferma',
                             'Campalto Bagaron': 'Terraferma',
                             'Dese': 'Terraferma',
                             'Torre Antica': 'Terraferma',
                             'Aeroporto': 'Terraferma',
                             'Tessera':'Terraferma',
                             'Campalto': 'Terraferma',
                             'other city': 'Terraferma'}

dic2023_d_listings['neighbourhood_cleansed'] = dic2023_d_listings['neighbourhood_cleansed'].replace(new_neighbourhoods_levels)
neighbourhoods_dummies = pd.get_dummies(dic2023_d_listings['neighbourhood_cleansed'], drop_first=True)
dic2023_d_listings = pd.concat([dic2023_d_listings, neighbourhoods_dummies], axis=1)

In [22]:
property_types_groupings = {
    'Entire rental unit': 'Entire Place',
    'Entire home': 'Entire Place',
    'Entire vacation home': 'Entire Place',
    'Entire serviced apartment': 'Entire Place',
    'Entire condo': 'Entire Place',
    'Entire loft': 'Entire Place',
    'Entire guesthouse': 'Entire Place',
    'Entire villa': 'Entire Place',
    'Entire townhouse': 'Entire Place',
    'Entire bungalow': 'Entire Place',
    'Entire guest suite': 'Entire Place',
    'Entire cottage': 'Entire Place',
    'Entire chalet': 'Entire Place',
    'Entire place': 'Entire Place',
    'Entire home/apt': 'Entire Place',
    'Private room in bed and breakfast': 'Private Room',
    'Private room in boat': 'Private Room',
    'Private room in rental unit': 'Private Room',
    'Private room in guest suite': 'Private Room',
    'Private room in villa': 'Private Room',
    'Private room in condo': 'Private Room',
    'Private room in home': 'Private Room',
    'Private room in guesthouse': 'Private Room',
    'Private room in serviced apartment': 'Private Room',
    'Private room in farm stay': 'Private Room',
    'Private room in loft': 'Private Room',
    'Private room in townhouse': 'Private Room',
    'Private room in vacation home': 'Private Room',
    'Private room in chalet': 'Private Room',
    'Private room in casa particular': 'Private Room',
    'Private room in pension': 'Private Room',
    'Private room in hostel': 'Private Room',
    'Shared room in bed and breakfast': 'Shared Room',
    'Shared room in rental unit': 'Shared Room',
    'Shared room in condo': 'Shared Room',
    'Shared room in home': 'Shared Room',
    'Shared room in hostel': 'Shared Room',
    'Castle': 'Unique Stays',
    'Boat': 'Unique Stays',
    'Houseboat': 'Unique Stays',
    'Tiny home': 'Unique Stays',
    'Casa particular': 'Unique Stays',
    'Room in bed and breakfast': 'Rooms in Commercial Establishments',
    'Room in boutique hotel': 'Rooms in Commercial Establishments',
    'Room in hotel': 'Rooms in Commercial Establishments',
    'Room in serviced apartment': 'Rooms in Commercial Establishments',
    'Room in aparthotel': 'Rooms in Commercial Establishments',
    'Room in hostel': 'Rooms in Commercial Establishments',
    'Room in heritage hotel': 'Rooms in Commercial Establishments',
    'Floor': 'Shared Room',
    'Private room': 'Private Room'
}

dic2023_d_listings['property_type'] = dic2023_d_listings['property_type'].replace(property_types_groupings)
categorial_property_type = CategoricalDtype(categories=['Shared Room',
                                                        'Private Room',
                                                        'Rooms in Commercial Establishments',
                                                        'Entire Place',
                                                        'Unique Stays'],
                                            ordered=True)
dic2023_d_listings.property_type = dic2023_d_listings.property_type.astype(categorial_property_type)

In [23]:
dic2023_d_listings.bathrooms_text = dic2023_d_listings.bathrooms_text.astype(CategoricalDtype(categories=["shared",
                                                                                                          "single",
                                                                                                          "private"],
                                                                                              ordered=True))

In [24]:
def remove_symbols(text):
    try:
        cleaned_text = re.sub(r'[$,]', '', text)
        return cleaned_text.strip()
    except:
        return None
dic2023_d_listings["price"] = dic2023_d_listings["price"].apply(remove_symbols).astype(float)

In [25]:
dic2023_d_listings.has_availability = dic2023_d_listings.has_availability.apply(lambda x: 1 if x=="t" else 0)

### Reviews


In [26]:
dic2023_d_listings["first_review"] = pd.to_datetime(dic2023_d_listings["first_review"])
dic2023_d_listings["last_review"] = pd.to_datetime(dic2023_d_listings["last_review"])


*To estimate the values of the reviews, I need to keep only numeric data in the dataset.
For this reason, I decided to use a simple Sentiment Analysis on the Description to at least
retain some information about the listing*

In [27]:
# create dataset for imputing reviews NAs and eliminate categorial variables
df_imputation = dic2023_d_listings.drop(["id",
                                         "name",
                                         "host_id",
                                         "host_since",
                                         "host_response_time",
                                         "host_verifications",     # to remove permanently
                                         "neighbourhood_cleansed", # to remove permanently
                                         "property_type",
                                         "bathrooms_text",
                                         "first_review",
                                         "last_review",
                                         "price"
                                         ], axis=1)

column_names = list(df_imputation.columns)

In [28]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    processed_text = ' '.join(lemmatized_tokens)
    return processed_text

analyzer = SentimentIntensityAnalyzer()
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    sentiment = scores['pos']
    return sentiment

In [29]:
df_imputation["neighborhood_overview"] = df_imputation["neighborhood_overview"].apply(preprocess_text)
df_imputation["neighborhood_overview"] = df_imputation["neighborhood_overview"].apply(get_sentiment)

df_imputation["host_about"] = df_imputation["host_about"].apply(preprocess_text)
df_imputation["host_about"] = df_imputation["host_about"].apply(get_sentiment)

In [30]:
imputer = KNNImputer(n_neighbors=5, weights="distance", metric="nan_euclidean", copy=False)
imputed_df = imputer.fit_transform(df_imputation)
imputed_df = pd.DataFrame(imputed_df, columns = column_names)

### Merge again datasets adding categorial columns

In [31]:
dropped_columns = ["id",
                   "name",
                   "host_id",
                   "host_since",
                   "host_response_time",
                   "property_type",
                   "bathrooms_text",
                   "first_review",
                   "last_review",
                   "neighborhood_overview", # add again string form
                   "host_about",            # add again string form
                   "price"
                   ]

imputed_df = imputed_df.drop(["neighborhood_overview",
                              "host_about"],
                             axis=1)

pre_concat_dic = dic2023_d_listings[dropped_columns]
clean_dic_listings = pd.concat([pre_concat_dic, imputed_df], axis=1)

In [32]:
# Now replace the missing values of:
# - first_review with the value of host_since of that host,
# - last_review with the max data for this dataset (31/12/2023)

clean_dic_listings.loc[clean_dic_listings["first_review"].isna(), "first_review"] = clean_dic_listings.loc[clean_dic_listings["first_review"].isna(), "host_since"]
clean_dic_listings.loc[clean_dic_listings["last_review"].isna(), "last_review"] = pd.to_datetime("26/12/2023", dayfirst=True)

clean_dic_listings.isnull().sum(axis=0)


## Reviews

In [35]:
dic2023_d_reviews = dic2023_d_reviews[["listing_id", "id", "date", "comments"]]